In [3]:
from keras.applications import inception_v3
from keras import backend as K

# do not model traning
K.set_learning_phase(0)

model = inception_v3.InceptionV3(weights='imagenet', include_top=False)

Using TensorFlow backend.


87916544/87910968 [==============================] - 3s 0us/step


In [0]:
layer_contributions = {
    'mixed2': 0.2,
    'mixed3': 3.,
    'mixed4': 2.,
    'mixed5': 1.5,
}

In [0]:
layer_dict = dict([(layer.name, layer) for layer in model.layers])

loss = K.variable(0.)
for layer_name in layer_contributions:
  coeff = layer_contributions[layer_name]

  # get layer output
  activation = layer_dict[layer_name].output 

  scaling = K.prod(K.cast(K.shape(activation), 'float32'))

  # 층 득성의 L2 노름 제곱을 손실에 추가 
  loss = loss + (coeff * K.sum(K.square(activation[:, 2:-2, 2:-2, :])) / scaling)

In [0]:
# 생성된 딥드림 이미지를 저장 
dream = model.input

# 손실에 대한 딥드림 이미지의 그래디언트를 계산 - 왜? 
grads = K.gradients(loss, dream)[0]

# 그래디언트를 정규화 - 이게 중요????
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

# 주어진 입력 이미지에서 손실과 그래디언트 값을 계산할 케라스 funtion 객체를 만듬 
outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
  outs = fetch_loss_and_grads([x])
  loss_value = outs[0]
  grad_value = outs[1]

  return loss_value, grad_value

# 경사 상승법을 여러번 반복? 
def gradient_axcent(x, iterations, step, max_loss=None):
  for i in range(iterations):
    loss_value, grad_value = eval_loss_and_grads(x)
    if max_loss is not None and loss_value > max_loss:
      break
    print('...', i,' 번째 손실 :', loss_value)
    x += step * grad_values
  return x

In [0]:
import numpy as np

step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20

max_loss = 10.

base_image_path = '/content/drive/My Drive/Colab Notebooks/creative_commons_elephant.jpg'

img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]
successive_shapes = [original_shape]

for i in range(1, num_octave):
  shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
  successive_shapes.append(shape)

successive_shapes = successive_shapes[::-1]

original_img = np.copy(img)
shrunk_origianl_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
  print('처리할 이미지 크기: ', shape)
  img = resize_img(img, shape)
  img = gradient_ascent(img, iterations = iterations, stop = step, max_loss = max_loss)

  upscaled_shrunk_original_img = resize_img(origianl_img, shape)

  same_size_origianl = resize_img(original_img, shape)

  lost_detail = same_size_original - upscaled_shrunk_original_img

  img += lost_detail
  shrunk_origianl_img = resize_img(original_img, shape)
  save_img(img, fname='dream_at_scale_' + str(shape) + '.png')

save_img(img, fanme='./satasets/final_dream.png')


